In [ ]:
from __future__ import absolute_import, division, print_function

import dolfin as dl
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# imrport hippylib as hp

import logging
logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)

np.random.seed(seed=1)

In [ ]:
# ndim = 2
nx = 32
ny = 32
mesh = dl.UnitSquareMesh(nx, ny)
Vh2 = dl.FunctionSpace(mesh, 'Lagrange', 2)
# Vh1 = dl.FunctionSpace(mesh, 'Lagrange', 1)
# Vh = [Vh2, Vh1, Vh2]
# print( "Number of dofs: STATE={0}, PARAMETER={1}, ADJOINT={2}".format(
#     Vh[hp.STATE].dim(), Vh[hp.PARAMETER].dim(), Vh[hp.ADJOINT].dim()) )

In [ ]:
def eval_on_grid(u, n=10):
    x = np.linspace(0,1,n)
    y = np.linspace(0,1,n)
    X,Y = np.meshgrid(x,y)
    xx = X.flatten()
    yy = Y.flatten()
    return np.array([u(x,y) for x, y in zip(xx,yy)])

In [ ]:
def f_expr(Vh, x,y):
    f = dl.interpolate(dl.Expression("std::exp(-pow(x[0]-{0},2)/0.1)*std::exp(-pow(x[1]-{1},2)/0.1)".format(x,y), degree=5), Vh)
    return f

def generate_f(Vh, flocs, weights):
    f = weights[0]*f_expr(Vh, flocs[0][0], flocs[0][1])
    for floc, weight in zip(flocs[1:], weights[1:]):
        f += weight*f_expr(Vh, floc[0], floc[1])
    return f

def solve_system(Vh, f, omega_const, r):
    def boundary(x, on_boundary):
        return on_boundary

    bc_state = dl.DirichletBC(Vh, dl.Constant(0.0), boundary)
    u = dl.Function(Vh)
    u_trial = dl.TrialFunction(Vh)
    u_test  = dl.TestFunction(Vh)
    omega = dl.Constant(omega_const)
    mtrue = dl.interpolate(dl.Expression('std::log( 60. - 40.*(pow(x[0] - 0.5,2) + pow(x[1] - 0.5,2) < pow({},2) ) )'.format(r), degree=5), Vh)
    
    a = (-dl.inner(dl.grad(u_trial), dl.grad(u_test)) + omega*omega*dl.inner(dl.exp(2*mtrue)*u_trial, u_test)) * dl.dx
    L = -f * u_test * dl.dx
    A,b = dl.assemble_system(a, L, bc_state)
    dl.solve(A, u.vector(), b)
    return u

In [ ]:
declist = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
flocs = np.array([[i,j] for i in declist for j in declist])
std = 0.005
weights = np.random.normal(1,std,len(flocs))

### Demonstrate that the two ways of defining the expression are equivalent.

In [ ]:
# f = generate_f(flocs,weights)

In [ ]:
# u = solve_system(f)

In [ ]:
# ug = eval_on_grid(u)

In [ ]:
# plt.contourf(ug.reshape(50,50))
# plt.colorbar()

## Alternative Generation Method

In [ ]:
omega_const = 0.5
r = 0.3
ulist = [solve_system(Vh2, generate_f(Vh2, [floc], [1]), omega_const, r) for floc in flocs]

In [ ]:
n = 120
U = np.array([eval_on_grid(u,n) for u in ulist]).T

In [ ]:
u2 = U @ np.array(weights)

Note that the error is close to machine-$\epsilon$

In [ ]:
def plot_state(u2,flocs):
    n = int(np.sqrt(u2.shape))
    x = np.linspace(0,1,n)
    y = np.linspace(0,1,n)
    plt.figure(figsize=(15,10))
    vmin=-0.7
    vmax=0.3
    plt.contourf(x, y, u2.reshape(n,n), alpha=0.8, vmin=vmin, vmax=vmax)
    plt.pcolor(x, y, u2.reshape(n,n), alpha=0.5, vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.scatter(flocs[:,0], flocs[:,1], marker='x', c='white', s=200, label='source locations')
    plt.legend(fontsize=18)
    plt.show()

In [ ]:
plot_state(u2, flocs)

In [ ]:
%%bash
git commit -am 'updates'

In [ ]:
def wrap_problem(nx=20, r=0.2, omega_const=0.5, std=0.005, n=50, num_f=4, seed=1):
    ndim = 2
    ny = nx
    mesh = dl.UnitSquareMesh(nx, ny)
    Vh2 = dl.FunctionSpace(mesh, 'Lagrange', 2)
#     Vh1 = dl.FunctionSpace(mesh, 'Lagrange', 1)
#     Vh = [Vh2, Vh1, Vh2]
#     print( "Number of dofs: STATE={0}, PARAMETER={1}, ADJOINT={2}".format(
#         Vh[STATE].dim(), Vh[PARAMETER].dim(), Vh[ADJOINT].dim()) )
    
    declist = np.linspace(0.1, 0.9, num_f)
    
    flocs = np.array([[i,j] for i in declist for j in declist])
    ulist = [solve_system(Vh2, generate_f(Vh2, [floc], [1]), omega_const, r) for floc in flocs] # solve system for each source
    
    np.random.seed(seed)
    weights = np.random.normal(0,std,len(flocs))
    U = np.array([eval_on_grid(u,n) for u in ulist]).T # evaluate on a mesh
    u2 = U @ np.array(weights) # sum state variables over all source terms.
    plot_state(u2,flocs)
    

In [ ]:
import ipywidgets as wd

In [ ]:
W_nx = wd.IntSlider(value=50, min=20, max=100, step=2, continuous_update=False)
# W_ny = wd.IntSlider(value=20, min=20, max=80, step=2, continuous_update=False)
W_r = wd.FloatSlider(value=0.2, min=0.1, max=0.4, step=0.05, continuous_update=False)
omega_options = [0.0125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0]
W_omega_const = wd.SelectionSlider(value=0.5, options=omega_options, continuous_update=False)
sd_options = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1.0]
W_std = wd.SelectionSlider(value=0.005, options=sd_options, continuous_update=False)
W_n = wd.IntSlider(value=10, min=50, max=100, step=2, continuous_update=False)
W_num_f = wd.IntSlider(value=1, min=1, max=20, step=1, continuous_update=False)
W_seed = wd.IntSlider(value=1, min=1, max=100, step=1, continuous_update=False)

kwds = {'nx': W_nx, 
        'r': W_r, 
        'omega_const': W_omega_const, 
        'std': W_std,
        'n': W_n,
        'num_f': W_num_f,
        'seed': W_seed
       }


In [ ]:
wd.interact_manual(wrap_problem, **kwds)